In [1]:
import MeCab
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import pymysql
import sqlalchemy
import re
import ast
from gensim.models import Word2Vec

In [2]:
df = pd.read_csv('esg_dataset_2021_cleansing.csv')
df['noun']  = df['noun'].map(lambda x : ast.literal_eval(x))
df.head(5)

,Unnamed: 0,id,title,url,press,date,content,noun
0,0,4522316,"올해는 ESG 정보공시 표준화 원년, 압박 더 거세진다",https://news.naver.com/main/read.naver?mode=LS...,머니투데이,2021.01.01.,() 산업·금융·법조계 ESG 전문가 대담]/사진제공=이미지투데이2020년은 ESG...,"[산업, 금융, 법조, ESG, 전문가, 대담, 사진, 제공, 이미지, ESG, 환..."
1,1,4726689,"서학개미 사로잡은 美 ESG ETF, 올해도 高高할까",https://news.naver.com/main/read.naver?mode=LS...,매일경제,2021.01.01.,() 작년 착한 투자에 30조 몰려인베스코솔라 年 급등美기업 IPO 열기도 뜨거워로...,"[작년, 투자, 려인, 베스코, 솔라, 年, 급등, 美, 기업, IPO, 로빈, 후..."
2,2,10271670,"[신년사]방문규 수은 행장 ""수출 6000억달러 선봉장될 것""",https://news.naver.com/main/read.naver?mode=LS...,뉴시스,2021.01.01.,() ESG 경영 강화...수출 중소중견기업 맞춤형 지원 방문규 수출입은행장 (사진...,"[ESG, 경영, 강화, 수출, 중소, 중견, 기업, 맞춤, 지원, 방문, 규, 수..."
3,3,576959,[韓산업 대전환 ①] 격변기의 시대…대전환 앞당겼다,https://news.naver.com/main/read.naver?mode=LS...,아이뉴스24,2021.01.01.,() 코로나 팬데믹에 뉴노멀 시대 활짝…대내외 경제 불안 속 미래 먹거리 찾기 총력...,"[코로나, 팬데믹, 뉴노멀, 시대, 대내외, 경제, 불안, 속, 미래, 먹거리, 총..."
4,4,4726714,"펀드사태 혼쭐난 금융권, 외부 `포청천` 영입",https://news.naver.com/main/read.naver?mode=LS...,매일경제,2021.01.01.,"() 4대 금융지주 인사 키워드는우리금융, 감사원 출신 신민철하나銀은 이인영 변호사...","[금융, 지주, 인사, 키워드, 금융, 감사원, 출신, 신민철, 銀, 이인영, 변호..."


In [3]:
tokenized_data = []
for noun_list in df['noun']:
    tokenized_data.append(noun_list)
tokenized_data[:3]

[['산업',
  '금융',
  '법조',
  'ESG',
  '전문가',
  '대담',
  '사진',
  '제공',
  '이미지',
  'ESG',
  '환경',
  '사회',
  '가치',
  '지배구조',
  '논의',
  '원년',
  '평가',
  '포털',
  '사이트',
  '다음',
  'Daum',
  'ESG',
  '키워드',
  '검색',
  '건수',
  '기준',
  '간',
  '전체',
  'ESG',
  '건수',
  '배',
  'ESG',
  '주목',
  '지난해',
  '전',
  '세계',
  '코로나',
  'COVID',
  '전염병',
  '경제',
  '시스템',
  '전반',
  '충격',
  '정부',
  '기업',
  '그간',
  '재무',
  '리스크',
  '요인',
  '관리',
  '필요',
  '절감',
  '주요',
  '기업',
  '금융',
  '사',
  'ESG',
  '전사',
  '리스크',
  '관리',
  '전략',
  '기획',
  '차원',
  '시작',
  '올해',
  '축년',
  '기업',
  '금융',
  '법조',
  '다양',
  '분야',
  '활동',
  'ESG',
  '전문가',
  '초빙',
  'ESG',
  '규제',
  '환경',
  '변화',
  '기업',
  '금융',
  '길',
  '주제',
  '대담',
  '진행',
  '이번',
  '대담',
  '권기준',
  'SK',
  '수',
  '펙',
  '협의회',
  'SV',
  '위원회',
  '부사장',
  '김성우',
  '김',
  '애',
  '장법',
  '사무소',
  '환경',
  '에너지',
  '연구',
  '소장',
  '우상현',
  'KB',
  '은행',
  'CIB',
  '기업',
  '투자',
  '금융',
  '전무',
  'KB',
  '증권',
  'IB',
  '투자',
  '은행',
  '부사장',
  '윤석모',
  '삼성증권',
  '리서치',
 

In [7]:
model5 = Word2Vec(sentences = tokenized_data, window = 5, min_count = 5, workers = 4, sg = 0)
model5.wv.vectors.shape
print('ESG :', model5.wv.most_similar("ESG", topn=20), '\n')
print('환경 :', model5.wv.most_similar("환경", topn=20), '\n')
print('지배 :', model5.wv.most_similar("지배", topn=20), '\n')
print('사회 :', model5.wv.most_similar("사회", topn=20), '\n')

ESG : [('지속가능', 0.7062663435935974), ('스지', 0.5952443480491638), ('내재', 0.5655710697174072), ('모범', 0.5621499419212341), ('전반', 0.5576635599136353), ('그간', 0.5558507442474365), ('자리', 0.554628312587738), ('CDR', 0.5481052398681641), ('지배구조', 0.544105589389801), ('책임', 0.5431168675422668), ('현황', 0.5425415635108948), ('시사점', 0.5409085154533386), ('건전', 0.5398516058921814), ('중점', 0.5375456809997559), ('원칙', 0.5353972911834717), ('앞', 0.5290119051933289), ('반영', 0.5274976491928101), ('CSR', 0.5254173278808594), ('키워드', 0.5249814391136169), ('전략', 0.5244607925415039)] 

환경 : [('전반', 0.5473460555076599), ('Governance', 0.5451825857162476), ('헌장', 0.5365786552429199), ('건전', 0.5358790755271912), ('화두', 0.5314939022064209), ('친환경', 0.525425910949707), ('책임', 0.5139530301094055), ('Social', 0.5130080580711365), ('G', 0.5037104487419128), ('가속', 0.499150812625885), ('인권', 0.48747968673706055), ('일환', 0.4866929054260254), ('박차', 0.4724774956703186), ('중심', 0.47086551785469055), ('확립', 0.4708486

In [7]:
model10 = Word2Vec(sentences = tokenized_data, window = 10, min_count = 5, workers = 4, sg = 0)
model10.wv.vectors.shape
print('ESG :', model10.wv.most_similar("ESG", topn=20), '\n')
print('환경 :', model10.wv.most_similar("환경", topn=20), '\n')
print('지배 :', model10.wv.most_similar("지배", topn=20), '\n')
print('사회 :', model10.wv.most_similar("사회", topn=20), '\n')

ESG : [('전반', 0.6164138913154602), ('반영', 0.6067644953727722), ('내재', 0.5941430330276489), ('전략', 0.5889125466346741), ('차원', 0.5552700161933899), ('대내외', 0.5488331913948059), ('책임', 0.5382051467895508), ('재무', 0.5330418944358826), ('스지', 0.5315617322921753), ('경영', 0.5231497287750244), ('현황', 0.5228894948959351), ('모범', 0.5221562385559082), ('최근', 0.5202509164810181), ('방향', 0.5109570026397705), ('자리', 0.5050156712532043), ('부합', 0.5046800374984741), ('실행', 0.5043748617172241), ('기업', 0.5043336153030396), ('전사', 0.5010535717010498), ('구체', 0.5002621412277222)] 

환경 : [('Governance', 0.5629689693450928), ('친환경', 0.5539735555648804), ('Social', 0.5490269064903259), ('화두', 0.5189526677131653), ('Environmental', 0.5137408971786499), ('건전', 0.5042656660079956), ('Environment', 0.49295705556869507), ('활동', 0.47916391491889954), ('전반', 0.4721404016017914), ('확립', 0.4633132219314575), ('실천', 0.4626006484031677), ('노력', 0.4560251533985138), ('윤리', 0.4556014835834503), ('영역', 0.4527364373207092

In [8]:
model15 = Word2Vec(sentences = tokenized_data, window = 15, min_count = 5, workers = 4, sg = 0)
model15.wv.vectors.shape
print('ESG :', model15.wv.most_similar("ESG", topn=20), '\n')
print('환경 :', model15.wv.most_similar("환경", topn=20), '\n')
print('지배 :', model15.wv.most_similar("지배", topn=20), '\n')
print('사회 :', model15.wv.most_similar("사회", topn=20), '\n')

ESG : [('내재', 0.5978147983551025), ('전반', 0.5908359885215759), ('반영', 0.5790857076644897), ('중요도', 0.5718863606452942), ('차원', 0.57069331407547), ('국내외', 0.5706164240837097), ('경영', 0.5483930110931396), ('CDR', 0.5475346446037292), ('전략', 0.5398035645484924), ('방향', 0.5300289392471313), ('기업', 0.5209841728210449), ('전사', 0.5181545615196228), ('대내외', 0.5173307657241821), ('개념', 0.5097604990005493), ('책임', 0.5094659924507141), ('체계', 0.5074191689491272), ('주요', 0.5044441819190979), ('실질', 0.4976465106010437), ('재무', 0.4921262264251709), ('모범', 0.4918989837169647)] 

환경 : [('친환경', 0.5489923357963562), ('Governance', 0.5463094711303711), ('Social', 0.5281028151512146), ('화두', 0.5034846663475037), ('Environmental', 0.4954868257045746), ('Environment', 0.48779061436653137), ('전반', 0.4614700376987457), ('실천', 0.46093830466270447), ('일환', 0.45398738980293274), ('투명', 0.45132192969322205), ('활동', 0.4483273923397064), ('책임', 0.4416530132293701), ('건전', 0.4374594986438751), ('확산', 0.4338813424110

In [10]:
model20 = Word2Vec(sentences = tokenized_data, window = 20, min_count = 5, workers = 4, sg = 0)
model20.wv.vectors.shape
print('ESG :', model20.wv.most_similar("ESG", topn=30), '\n')
print('환경 :', model20.wv.most_similar("환경", topn=30), '\n')
print('지배 :', model20.wv.most_similar("지배", topn=30), '\n')
print('사회 :', model20.wv.most_similar("사회", topn=30), '\n')

ESG : [('전반', 0.6380619406700134), ('차원', 0.586173951625824), ('내재', 0.5861560106277466), ('반영', 0.583159863948822), ('중요도', 0.5758392810821533), ('전략', 0.5543038845062256), ('국내외', 0.5447744131088257), ('기업', 0.5376449823379517), ('화두', 0.535119354724884), ('현황', 0.5222923755645752), ('재무', 0.5219850540161133), ('방향', 0.5177162289619446), ('대내외', 0.5163207054138184), ('경영', 0.5147543549537659), ('실행', 0.5096827745437622), ('CDR', 0.5060709714889526), ('최근', 0.5035495162010193), ('개념', 0.5022788047790527), ('모범', 0.4981435239315033), ('부합', 0.49520546197891235), ('책임', 0.4888954758644104), ('중요', 0.48377692699432373), ('쟁점', 0.48289817571640015), ('구체', 0.48149722814559937), ('체계', 0.48134157061576843), ('이해관계인', 0.4797806739807129), ('김상준', 0.4785592555999756), ('정립', 0.47620394825935364), ('사례', 0.4750491678714752), ('스지', 0.4742598533630371)] 

환경 : [('Social', 0.5789974927902222), ('Governance', 0.5779455304145813), ('Environment', 0.5694883465766907), ('친환경', 0.5517378449440002), 

In [11]:
model20 = Word2Vec(sentences = tokenized_data, window = 20, min_count = 5, workers = 4, sg = 0)
model20.wv.vectors.shape
print('ESG :', model20.wv.most_similar("ESG", topn=40), '\n')
print('환경 :', model20.wv.most_similar("환경", topn=40), '\n')
print('지배 :', model20.wv.most_similar("지배", topn=40), '\n')
print('사회 :', model20.wv.most_similar("사회", topn=40), '\n')

ESG : [('반영', 0.6157254576683044), ('전반', 0.6117823123931885), ('내재', 0.5939911603927612), ('차원', 0.5636358857154846), ('전략', 0.5589246153831482), ('기업', 0.547243058681488), ('CDR', 0.5437421202659607), ('경영', 0.5409958362579346), ('현황', 0.5272204875946045), ('부합', 0.5192704200744629), ('실행', 0.5178439021110535), ('재무', 0.517108678817749), ('송재형', 0.5154074430465698), ('실질', 0.5152561664581299), ('도출', 0.5145314335823059), ('책임', 0.5142039060592651), ('모범', 0.507899820804596), ('주요', 0.5071188807487488), ('전사', 0.5039395093917847), ('수립', 0.5023794770240784), ('구체', 0.501733124256134), ('요소', 0.49606403708457947), ('중요', 0.4940752685070038), ('중요도', 0.492964506149292), ('최근', 0.4927767217159271), ('국내외', 0.4911390542984009), ('화두', 0.49086201190948486), ('지표', 0.4878841042518616), ('방향', 0.4876262843608856), ('계량화', 0.48599687218666077), ('체계', 0.4842815101146698), ('현주소', 0.48391327261924744), ('꼬리표', 0.4815996289253235), ('성과', 0.47541558742523193), ('관련', 0.4752984344959259), ('마련',

In [5]:
model20 = Word2Vec(sentences = tokenized_data, window = 30, min_count = 75, workers = 4, sg = 0)
model20.wv.vectors.shape
print('ESG :', model20.wv.most_similar("ESG", topn=100), '\n')
print('환경 :', model20.wv.most_similar("환경", topn=100), '\n')
print('사회 :', model20.wv.most_similar("사회", topn=100), '\n')
print('지배구조 :', model20.wv.most_similar("지배", topn=100), '\n')

ESG : [('화두', 0.5809404253959656), ('경영', 0.5666679739952087), ('전반', 0.5297560095787048), ('지속가능', 0.5139879584312439), ('차원', 0.49958404898643494), ('책임', 0.49161237478256226), ('내재', 0.4808666408061981), ('Governance', 0.47853732109069824), ('국내외', 0.47296142578125), ('전략', 0.46227890253067017), ('필수', 0.4619462788105011), ('재무', 0.45786795020103455), ('반영', 0.45339322090148926), ('요소', 0.4531286656856537), ('부상', 0.44425979256629944), ('전담', 0.4436569809913635), ('기업', 0.44207897782325745), ('Environment', 0.4408644139766693), ('Social', 0.44053760170936584), ('수립', 0.4242459237575531), ('주요', 0.42145836353302), ('원년', 0.4212510585784912), ('산하', 0.418760746717453), ('정립', 0.41212189197540283), ('SRI', 0.4041256010532379), ('지배구조', 0.4035126268863678), ('실질', 0.39937129616737366), ('개념', 0.39617863297462463), ('고려', 0.3942958414554596), ('방향', 0.3919641971588135), ('최근', 0.39128217101097107), ('신설', 0.39111238718032837), ('실행', 0.3884941637516022), ('중요', 0.3877091109752655), ('모범'

In [6]:
df = pd.DataFrame(model20.wv.most_similar("ESG", topn=100))
df1 = pd.DataFrame(model20.wv.most_similar("환경", topn=100))
df2 = pd.DataFrame(model20.wv.most_similar("사회", topn=100))
df3 = pd.DataFrame(model20.wv.most_similar("지배구조", topn=100))

In [7]:
a = pd.concat([df,df1])
a = pd.concat([a,df2])
a = pd.concat([a,df3])
print(a)

       0         1
0     화두  0.580940
1     경영  0.566668
2     전반  0.529756
3   지속가능  0.513988
4     차원  0.499584
..   ...       ...
95    준수  0.202423
96   선언문  0.202142
97    자리  0.201010
98   안전망  0.200477
99    일환  0.199569

[400 rows x 2 columns]


In [8]:
a.to_csv('article_embed_top100.csv',encoding = 'utf-8-sig')

In [1]:
model20 = Word2Vec(sentences = tokenized_data, window = 10, min_count = 75, workers = 4, sg = 0)
model20.wv.vectors.shape
print('ESG :', model20.wv.most_similar("ESG", topn=40), '\n')
print('환경 :', model20.wv.most_similar("환경", topn=40), '\n')
print('지배 :', model20.wv.most_similar("지배", topn=40), '\n')
print('사회 :', model20.wv.most_similar("사회", topn=40), '\n')

NameError: name 'Word2Vec' is not defined

In [2]:
model20 = Word2Vec(sentences = tokenized_data, window = 20, min_count = 75, workers = 4, sg = 0)
model20.wv.vectors.shape
print('ESG :', model20.wv.most_similar("ESG", topn=30), '\n')
print('환경 :', model20.wv.most_similar("환경", topn=30), '\n')
print('지배 :', model20.wv.most_similar("지배", topn=30), '\n')
print('사회 :', model20.wv.most_similar("사회", topn=30), '\n')

NameError: name 'Word2Vec' is not defined

In [26]:
model20 = Word2Vec(sentences = tokenized_data, window = 20, min_count = 75, workers = 4, sg = 0)
model20.wv.vectors.shape
print('ESG :', model20.wv.most_distance("ESG", topn=200), '\n')
#print('환경 :', model20.wv.most_similar("환경", topn=40), '\n')
#print('지배 :', model20.wv.most_similar("지배", topn=40), '\n')
#print('사회 :', model20.wv.most_similar("사회", topn=40), '\n')

AttributeError: 'KeyedVectors' object has no attribute 'most_distance'

In [28]:
model20 = Word2Vec(sentences = tokenized_data, window = 10, min_count = 75, workers = 4, sg = 0)

article_esg = model20.wv.most_similar("ESG", topn=300)
print(article_esg[0][0])

article_esg_word = []
for i in range(len(article_esg)):
    article_esg_word.append(article_esg[i][0])
print(article_esg_word)


내재
['내재', '전반', '화두', '반영', '경영', '재무', '부상', 'Governance', 'Environment', 'Social', '책임', '전략', '정립', '실질', '차원', '거버넌스', '스지', '신설', '중요', '전사', '요소', '국내외', '현황', '가속', '최근', '체계', '방향', 'CSR', '의지', '키워드', '핵심', '모범', '원년', '역할', '확산', '투명', '원칙', '전담', '사항', '적극', '실행', '산하', '성과', '개념', '필수', '부합', '수립', '구체', '확립', '세부', '과제', '가이드라인', '주요', '기업', 'SRI', '지속', '트렌드', '지표', '발간', '이행', '앞', '강조', '말', '발족', '사회', '자리', '가운데', '위원회', '환경', '평가', '이슈', '고려', '박차', '워싱', '윤리', '패러다임', '본격', '업계', '의사', '요구', '이사회', '계열사', '추세', '보고', '논의', '준법', '부서', '증권사', '관점', '글로벌', '선도', '제고', '금융', '국내', '철학', '헌장', '보고서', '공통', '제정', '방안', '선포', '사', '부각', '연기금', '자리매김', '제시', '감독', '강화', '지배', '조직', '도출', '중심', '영역', '실무', '내용', '대외', '운용', '로드맵', '구조', '인식', '관련', '선언', '투자자', '기업지배구조', '심의', '배경', '자문', '개편', '마련', '공감대', '이념', '증권', '여신', '방침', '한편', '리포트', '중점', '결정', '세미나', '토대', '경영진', '체제', '움직임', '실천', '정착', '신한', '취지', '발표', '가치', '행보', '노력', 'KPMG', '출범', '급변', '개선', '대응', '이니셔티브'